In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Premier League Player Standard Stats 2023-2024.csv', encoding='ISO-8859-1')

In [3]:
print(df.head(10))

               Player   Nation    Pos            Squad     Age  Born  \
0          Max Aarons  eng ENG     DF      Bournemouth  24-111  2000   
1  Bénie Adama Traore   ci CIV  FW,MF    Sheffield Utd  21-146  2002   
2         Tyler Adams   us USA     MF      Bournemouth  25-070  1999   
3    Tosin Adarabioyo  eng ENG     DF           Fulham  26-213  1997   
4      Elijah Adebayo  eng ENG     FW       Luton Town  26-108  1998   
5       Simon Adingra   ci CIV     FW         Brighton  22-114  2002   
6        Nayef Aguerd   ma MAR     DF         West Ham  28-025  1996   
7    Brandon Aguilera   cr CRC     FW  Nott'ham Forest  20-301  2003   
8    Naouirou Ahamada   fr FRA  MF,FW   Crystal Palace  22-026  2002   
9    Anel Ahmedhodi?   ba BIH     DF    Sheffield Utd  25-029  1999   

   Matches Played  Starts Total Min  90s Played  ...  Goals per 90  \
0              17      12     1,152        12.8  ...          0.00   
1               8       3       387         4.3  ...          0.00 

In [4]:
df.fillna(df.mean(), inplace=True)
df.drop_duplicates(inplace=True)

C:\Users\ehoml\AppData\Local\Temp\ipykernel_21156\159524517.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(df.mean(), inplace=True)


In [5]:
df['Nation'] = df['Nation'].str.replace(r'\b[a-z]+', '', regex=True).str.strip()
print(df['Nation'].head())

0    ENG
1    CIV
2    USA
3    ENG
4    ENG
Name: Nation, dtype: object


In [6]:
pip install requests beautifulsoup4 pandas

Note: you may need to restart the kernel to use updated packages.


In [7]:
import requests
from bs4 import BeautifulSoup

In [8]:
url = 'https://fbref.com/en/comps/9/wages/Premier-League-Wages'


response = requests.get(url)


if response.status_code == 200:
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all tables on the page
    tables = soup.find_all('table')
    
    # The player wages are expected to be in the second table on the page
    player_wages_table_html = str(tables[1])
    
    # Read the table into a pandas dataframe
    player_wages_df = pd.read_html(player_wages_table_html)[0]
    
    # Clean up the DataFrame if necessary
    # Flatten the MultiIndex headers if the table has one
    if isinstance(player_wages_df.columns, pd.MultiIndex):
        player_wages_df.columns = ['_'.join(col).strip() for col in player_wages_df.columns.values]
    
    # Save the dataframe to a CSV file
    player_wages_df.to_csv('player_wages_2023_2024.csv', index=False)
    
    # Print the first few rows
    print(player_wages_df.head())
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

   Rk           Player  Nation Pos            Squad  Age  \
0   1  Kevin De Bruyne  be BEL  MF  Manchester City   32   
1   2   Erling Haaland  no NOR  FW  Manchester City   23   
2   3         Casemiro  br BRA  MF   Manchester Utd   31   
3   4    Mohamed Salah  eg EGY  FW        Liverpool   31   
4   5   Raphaël Varane  fr FRA  DF   Manchester Utd   30   

                      Weekly Wages                              Annual Wages  \
0  £ 400,000 (€ 464,794, $495,537)  £ 20,800,000 (€ 24,169,284, $25,767,947)   
1  £ 375,000 (€ 435,744, $464,566)  £ 19,500,000 (€ 22,658,704, $24,157,450)   
2  £ 350,000 (€ 406,695, $433,595)  £ 18,200,000 (€ 21,148,123, $22,546,954)   
3  £ 350,000 (€ 406,695, $433,595)  £ 18,200,000 (€ 21,148,123, $22,546,954)   
4  £ 340,000 (€ 395,075, $421,207)  £ 17,680,000 (€ 20,543,891, $21,902,755)   

  Notes  
0   NaN  
1   NaN  
2   NaN  
3   NaN  
4   NaN  


In [9]:
new_df = df.merge(player_wages_df, on='Player', how='left')

In [10]:
print(new_df)

                  Player Nation_x  Pos_x        Squad_x   Age_x  Born  \
0             Max Aarons      ENG     DF    Bournemouth  24-111  2000   
1     Bénie Adama Traore      CIV  FW,MF  Sheffield Utd  21-146  2002   
2            Tyler Adams      USA     MF    Bournemouth  25-070  1999   
3       Tosin Adarabioyo      ENG     DF         Fulham  26-213  1997   
4         Elijah Adebayo      ENG     FW     Luton Town  26-108  1998   
..                   ...      ...    ...            ...     ...   ...   
560       Nicolò Zaniolo      ITA  FW,MF    Aston Villa  24-297  1999   
561        Anass Zaroury      MAR  FW,MF        Burnley  23-169  2000   
562  Oleksandr Zinchenko      UKR     DF        Arsenal  27-131  1996   
563           Kurt Zouma      FRA     DF       West Ham  29-180  1994   
564      Martin Ødegaard      NOR     MF        Arsenal  25-129  1998   

     Matches Played  Starts Total Min  90s Played  ...  Non PK xG per 90  \
0                17      12     1,152        12

In [11]:
# The URL for the Big 5 European Leagues Stats
url = 'https://fbref.com/en/comps/Big5/misc/players/Big-5-European-Leagues-Stats'


response = requests.get(url)

if response.status_code == 200:
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # The table of interest
    table = soup.find_all('table')[0]
    
    # Read the table into a pandas dataframe
    player_stats_df = pd.read_html(str(table))[0]
    
   
    player_stats_df.columns = [col[-1] for col in player_stats_df.columns.values]

    # Filter the DataFrame for only Premier League players
    premier_league_players_df = player_stats_df[player_stats_df['Comp'] == 'Premier League']

   
    premier_league_players_df.to_csv('premier_league_player_misc_stats.csv', index=False)
    
    # Print the first few rows
    print(premier_league_players_df.head())

else:
    print(f"Failed to retrieve data: status code {response.status_code}")

Empty DataFrame
Columns: [Rk, Player, Nation, Pos, Squad, Comp, Age, Born, 90s, CrdY, CrdR, 2CrdY, Fls, Fld, Off, Crs, Int, TklW, PKwon, PKcon, OG, Recov, Won, Lost, Won%, Matches]
Index: []

[0 rows x 26 columns]


In [12]:
print(player_stats_df['Comp'].unique())

['eng Premier League' 'de Bundesliga' 'fr Ligue 1' 'es La Liga'
 'it Serie A' 'Comp']


In [13]:
premier_league_mstats_df = player_stats_df[player_stats_df['Comp'].str.contains('eng Premier League', na=False)]

In [14]:
print(premier_league_mstats_df.head())

    Rk              Player   Nation    Pos          Squad                Comp  \
0    1          Max Aarons  eng ENG     DF    Bournemouth  eng Premier League   
14  15  Bénie Adama Traore   ci CIV  FW,MF  Sheffield Utd  eng Premier League   
17  18         Tyler Adams   us USA     MF    Bournemouth  eng Premier League   
20  21    Tosin Adarabioyo  eng ENG     DF         Fulham  eng Premier League   
21  22      Elijah Adebayo  eng ENG     FW     Luton Town  eng Premier League   

       Age  Born   90s CrdY  ... Int TklW PKwon PKcon OG Recov Won Lost  Won%  \
0   24-114  2000  12.8    1  ...   6   19     0     1  0    73   5   10  33.3   
14  21-149  2002   4.3    0  ...   1    2     0     0  0    22   2   18  10.0   
17  25-073  1999   1.2    0  ...   4    3     0     0  0     7   2    3  40.0   
20  26-216  1997  18.0    2  ...  25   11     0     0  0    43  56   28  66.7   
21  26-111  1998  12.9    1  ...   5    0     0     0  0    30  38   38  50.0   

    Matches  
0   Matches 

In [15]:
premier_league_mstats_df.to_csv('premier_league_mstats.csv', index=False)

In [16]:
df_combined = df.merge(premier_league_mstats_df, on='Player', how='outer', suffixes=('_standard', '_misc'))
df_final_combined = df_combined.merge(player_wages_df, on='Player', how='outer')

In [17]:
df_final_combined

,Player,Nation_standard,Pos_standard,Squad_standard,Age_standard,Born_standard,Matches Played,Starts,Total Min,90s Played,...,Won%,Matches,Rk_y,Nation,Pos,Squad,Age,Weekly Wages,Annual Wages,Notes
0,Max Aarons,ENG,DF,Bournemouth,24-111,2000.0,17.0,12.0,"1,152",12.8,...,33.3,Matches,338.0,eng ENG,DF,Bournemouth,23.0,"£ 35,000 (€ 40,669, $43,360)","£ 1,820,000 (€ 2,114,812, $2,254,695)",Unverified estimation
1,Bénie Adama Traore,CIV,"FW,MF",Sheffield Utd,21-146,2002.0,8.0,3.0,387,4.3,...,10.0,Matches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Tyler Adams,USA,MF,Bournemouth,25-070,1999.0,2.0,1.0,110,1.2,...,40.0,Matches,202.0,us USA,MF,Bournemouth,24.0,"£ 60,000 (€ 69,719, $74,331)","£ 3,120,000 (€ 3,625,393, $3,865,192)",Unverified estimation
3,Tosin Adarabioyo,ENG,DF,Fulham,26-213,1997.0,20.0,18.0,"1,617",18.0,...,66.7,Matches,327.0,eng ENG,DF,Fulham,25.0,"£ 40,000 (€ 46,479, $49,554)","£ 2,080,000 (€ 2,416,928, $2,576,795)",NaN
4,Elijah Adebayo,ENG,FW,Luton Town,26-108,1998.0,23.0,13.0,"1,162",12.9,...,50.0,Matches,500.0,eng ENG,FW,Luton Town,25.0,"£ 12,500 (€ 14,525, $15,486)","£ 650,000 (€ 755,290, $805,248)",Unverified estimation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681,Denis Franchi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,586.0,it ITA,GK,Burnley,20.0,"£ 3,000 (€ 3,486, $3,717)","£ 156,000 (€ 181,270, $193,260)",Unverified estimation
682,Ryan Trevitt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,588.0,NaN,CM,Brentford,20.0,"£ 2,500 (€ 2,905, $3,097)","£ 130,000 (€ 151,058, $161,050)",Unverified estimation
683,Elliot Thorpe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,589.0,wls WAL,CM,Luton Town,23.0,"£ 2,500 (€ 2,905, $3,097)","£ 130,000 (€ 151,058, $161,050)",Unverified estimation
684,Gavin Kilkenny,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,590.0,ie IRL,MF,Bournemouth,23.0,"£ 1,500 (€ 1,743, $1,858)","£ 78,000 (€ 90,635, $96,630)",Unverified estimation


In [18]:
column_names = df_final_combined.columns.tolist()
print(column_names)

['Player', 'Nation_standard', 'Pos_standard', 'Squad_standard', 'Age_standard', 'Born_standard', 'Matches Played', 'Starts', 'Total Min', '90s Played', 'Goals', 'Ast', 'G+A', 'Non-PK Goals', 'PK', 'PKattempts', 'Crd-Yellow', 'Crd-Red', 'xGoals', 'Non-PK xG', 'xAG', 'npxG+xAG', 'Progressive Carries', 'Progressive Passes', 'Progressive Passes Rec', 'Goals per 90', 'Assists per 90', 'Goals +Assists  per 90', 'Non-PK Goals per 90', 'Non-PK Goals/Assists per 90', 'xG per 90', 'xAG per 90', 'xG+xAG per 90', 'Non PK xG per 90', 'xNon PK Goals + Assisted Goals per 90', 'Rk_x', 'Nation_misc', 'Pos_misc', 'Squad_misc', 'Comp', 'Age_misc', 'Born_misc', '90s', 'CrdY', 'CrdR', '2CrdY', 'Fls', 'Fld', 'Off', 'Crs', 'Int', 'TklW', 'PKwon', 'PKcon', 'OG', 'Recov', 'Won', 'Lost', 'Won%', 'Matches', 'Rk_y', 'Nation', 'Pos', 'Squad', 'Age', 'Weekly Wages', 'Annual Wages', 'Notes']


In [19]:
cleaned_df = pd.DataFrame(index=df_final_combined.index)
for col in df_final_combined.columns:
    if col not in cleaned_df.columns: 
        cleaned_df[col] = df_final_combined[col]

df_final_combined = cleaned_df

# Display the DataFrame to verify that duplicate columns have been removed
print(df_final_combined)

# Save the cleaned DataFrame to a new CSV file
df_final_combined.to_csv('2023-2024 Premier League Player Stats and Wages.csv', index=False)

                 Player Nation_standard Pos_standard Squad_standard  \
0            Max Aarons             ENG           DF    Bournemouth   
1    Bénie Adama Traore             CIV        FW,MF  Sheffield Utd   
2           Tyler Adams             USA           MF    Bournemouth   
3      Tosin Adarabioyo             ENG           DF         Fulham   
4        Elijah Adebayo             ENG           FW     Luton Town   
..                  ...             ...          ...            ...   
681       Denis Franchi             NaN          NaN            NaN   
682        Ryan Trevitt             NaN          NaN            NaN   
683       Elliot Thorpe             NaN          NaN            NaN   
684      Gavin Kilkenny             NaN          NaN            NaN   
685         Louie Marsh             NaN          NaN            NaN   

    Age_standard  Born_standard  Matches Played  Starts Total Min  90s Played  \
0         24-111         2000.0            17.0    12.0     1,152 

In [20]:
import pandas as pd

df_final_combined = pd.read_csv('2023-2024 Premier League Player Stats and Wages.csv')

# Columns to be removed 
columns_to_remove = [
    'Rk_x', 'Nation_misc', 'Position_misc', 'Squad_misc', 'Comp', 
    'Age_misc', 'Born_misc', '90s', 'CrdY', 'Crd R', 'Matches', 
    'Rk_y', 'Nation', 'Pos', 'Squad', 'Age', 'Notes'
]


df_final_combined = df_final_combined.drop(columns=[col for col in columns_to_remove if col in df_final_combined.columns])

# Rename columns by removing '_standard' from the column names
df_final_combined.rename(columns=lambda x: x.replace('_standard', ''), inplace=True)

# Drop rows where all elements are NaN to clean up the DataFrame
df_final_combined.dropna(how='all', inplace=True)

df_final_combined = df_final_combined[df_final_combined['Player'].notna()]

# Remove rows with duplicate player names, keeping the first occurrence
df_final_combined = df_final_combined.drop_duplicates(subset='Player', keep='first')

# Save the cleaned DataFrame to a new CSV file
cleaned_csv_path = '2023-2024_Premier_League_Player_Stats_and_Wages.csv'
df_final_combined.to_csv(cleaned_csv_path, index=False)


print(df_final_combined.head())

cleaned_csv_path

               Player Nation    Pos          Squad     Age    Born  \
0          Max Aarons    ENG     DF    Bournemouth  24-111  2000.0   
1  Bénie Adama Traore    CIV  FW,MF  Sheffield Utd  21-146  2002.0   
2         Tyler Adams    USA     MF    Bournemouth  25-070  1999.0   
3    Tosin Adarabioyo    ENG     DF         Fulham  26-213  1997.0   
4      Elijah Adebayo    ENG     FW     Luton Town  26-108  1998.0   

   Matches Played  Starts Total Min  90s Played  ...  TklW  PKwon  PKcon   OG  \
0            17.0    12.0     1,152        12.8  ...  19.0    0.0    1.0  0.0   
1             8.0     3.0       387         4.3  ...   2.0    0.0    0.0  0.0   
2             2.0     1.0       110         1.2  ...   3.0    0.0    0.0  0.0   
3            20.0    18.0     1,617        18.0  ...  11.0    0.0    0.0  0.0   
4            23.0    13.0     1,162        12.9  ...   0.0    0.0    0.0  0.0   

   Recov   Won  Lost  Won%                  Weekly Wages  \
0   73.0   5.0  10.0  33.3  £ 35

'2023-2024_Premier_League_Player_Stats_and_Wages.csv'

In [22]:
df_final_combined.to_csv('The 2023-2024 Premier League Player Stats and Wages.csv', index=False)

In [23]:
url = 'https://fbref.com/en/comps/Big5/playingtime/players/Big-5-European-Leagues-Stats'
response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table')
    new_data_df = pd.read_html(str(table))[0]

new_data_df.columns = [col[-1] if isinstance(col, tuple) else col for col in new_data_df.columns.values]

# Filter for Premier League players
premier_league_data_df = new_data_df[new_data_df['Comp'].str.contains('Premier League', na=False)]
combined_df = pd.merge(df_final_combined, premier_league_data_df, on='Player', how='outer', suffixes=('', '_new'))
combined_df.to_csv('Final 2023-2024 Premier League Player Stats and Wages.csv', index=False)

print(combined_df.head())

               Player Nation    Pos          Squad     Age    Born  \
0          Max Aarons    ENG     DF    Bournemouth  24-111  2000.0   
1  Bénie Adama Traore    CIV  FW,MF  Sheffield Utd  21-146  2002.0   
2         Tyler Adams    USA     MF    Bournemouth  25-070  1999.0   
3    Tosin Adarabioyo    ENG     DF         Fulham  26-213  1997.0   
4      Elijah Adebayo    ENG     FW     Luton Town  26-108  1998.0   

   Matches Played  Starts Total Min  90s Played  ...  onGA  +/-  +/-90  \
0            17.0    12.0     1,152        12.8  ...    31  -21  -1.64   
1             8.0     3.0       387         4.3  ...    13  -11  -2.56   
2             2.0     1.0       110         1.2  ...     1   +2  +1.64   
3            20.0    18.0     1,617        18.0  ...    25   +3  +0.17   
4            23.0    13.0     1,162        12.9  ...    28   -3  -0.23   

   On-Off  onxG  onxGA  xG+/-  xG+/-90  On-Off  Matches  
0   -2.11  12.6   26.2  -13.5    -1.06   -1.65  Matches  
1   -0.94   3.1   

In [28]:
combined_df = pd.read_csv('Final 2023-2024 Premier League Player Stats and Wages.csv')


cleaned_df = pd.DataFrame(index=combined_df.index)


for col in combined_df.columns:
    if col not in cleaned_df.columns:
        cleaned_df[col] = combined_df[col]

combined_df = cleaned_df


cleaned_df.to_csv('cleaned_Final_2023-2024_Premier_League_Player_Stats_and_Wages.csv', index=False)

print(cleaned_df.head())

               Player Nation    Pos          Squad     Age    Born  \
0          Max Aarons    ENG     DF    Bournemouth  24-111  2000.0   
1  Bénie Adama Traore    CIV  FW,MF  Sheffield Utd  21-146  2002.0   
2         Tyler Adams    USA     MF    Bournemouth  25-070  1999.0   
3    Tosin Adarabioyo    ENG     DF         Fulham  26-213  1997.0   
4      Elijah Adebayo    ENG     FW     Luton Town  26-108  1998.0   

   Matches Played  Starts Total Min  90s Played  ...  onGA   +/-  +/-90  \
0            17.0    12.0     1,152        12.8  ...  31.0 -21.0  -1.64   
1             8.0     3.0       387         4.3  ...  13.0 -11.0  -2.56   
2             2.0     1.0       110         1.2  ...   1.0   2.0   1.64   
3            20.0    18.0     1,617        18.0  ...  25.0   3.0   0.17   
4            23.0    13.0     1,162        12.9  ...  28.0  -3.0  -0.23   

   On-Off  onxG  onxGA  xG+/-  xG+/-90  On-Off.1  Matches  
0   -2.11  12.6   26.2  -13.5    -1.06     -1.65  Matches  
1   -0.9

In [31]:
cleaned_df = pd.read_csv('cleaned_Final_2023-2024_Premier_League_Player_Stats_and_Wages.csv')

# Columns to be removed
columns_to_remove = [
    'Rk', 'Nation_new', 'Pos_new', 'Squad_new', 'Comp', 
    'Age_new', 'Born_new', 'MP', 'Min', 'MN/MP', 'Matches', 
    'Starts_new', 'Pos_misc', '90s'
]

# Remove columns
cleaned_df = cleaned_df.drop(columns=[col for col in columns_to_remove if col in cleaned_df.columns])

# Drop rows where all elements are NaN 
cleaned_df.dropna(how='all', inplace=True)

cleaned_df = cleaned_df[cleaned_df['Player'].notna()]

# Remove rows with duplicate player names, keeping the first occurrence
cleaned_df = cleaned_df.drop_duplicates(subset='Player', keep='first')


cleaned_csv_path = 'Complete 2023-2024_Premier_League_Player_Stats_and_Wages.csv'
cleaned_df.to_csv(cleaned_csv_path, index=False)


print(cleaned_df.head())

cleaned_csv_path

               Player Nation    Pos          Squad     Age    Born  \
0          Max Aarons    ENG     DF    Bournemouth  24-111  2000.0   
1  Bénie Adama Traore    CIV  FW,MF  Sheffield Utd  21-146  2002.0   
2         Tyler Adams    USA     MF    Bournemouth  25-070  1999.0   
3    Tosin Adarabioyo    ENG     DF         Fulham  26-213  1997.0   
4      Elijah Adebayo    ENG     FW     Luton Town  26-108  1998.0   

   Matches Played  Starts Total Min  90s Played  ...   onG  onGA   +/-  +/-90  \
0            17.0    12.0     1,152        12.8  ...  10.0  31.0 -21.0  -1.64   
1             8.0     3.0       387         4.3  ...   2.0  13.0 -11.0  -2.56   
2             2.0     1.0       110         1.2  ...   3.0   1.0   2.0   1.64   
3            20.0    18.0     1,617        18.0  ...  28.0  25.0   3.0   0.17   
4            23.0    13.0     1,162        12.9  ...  25.0  28.0  -3.0  -0.23   

   On-Off  onxG  onxGA  xG+/-  xG+/-90  On-Off.1  
0   -2.11  12.6   26.2  -13.5    -1.06   

'Complete 2023-2024_Premier_League_Player_Stats_and_Wages.csv'

In [2]:
import pandas as pd
df = pd.read_csv('Complete 2023-2024_Premier_League_Player_Stats_and_Wages.csv')

rows, columns = df.shape

print(f'The dataset contains {rows} rows (samples) and {columns} columns (features).')

The dataset contains 555 rows (samples) and 70 columns (features).


In [3]:
df.describe()

,Age,Born,Matches Played,Starts,90s Played,Goals,Ast,G+A,Non-PK Goals,PK,...,onG,onGA,+/-,+/-90,On-Off,onxG,onxGA,xG+/-,xG+/-90,On-Off.1
count,555.000000,555.000000,555.000000,555.000000,555.000000,555.000000,555.000000,555.000000,555.000000,555.000000,...,541.000000,541.000000,541.000000,541.000000,533.000000,540.000000,540.000000,540.000000,540.000000,532.000000
mean,25.794595,1997.536937,17.668468,13.003604,12.975315,1.818018,1.306306,3.124324,1.682883,0.135135,...,21.530499,21.428835,0.101664,-0.342551,-0.278593,20.351852,20.297778,0.055741,-0.318556,-0.228477
std,4.411706,4.408720,10.160357,10.420577,9.926387,3.062119,2.055163,4.599565,2.764586,0.596114,...,18.466624,16.919116,13.701136,3.515779,3.487013,16.724890,15.866855,10.574748,2.689361,3.003040
min,17.000000,1984.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,-49.000000,-30.000000,-30.530000,0.000000,0.000000,-32.400000,-39.590000,-40.670000
25%,22.000000,1994.000000,9.000000,3.000000,3.950000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,7.000000,-6.000000,-0.780000,-0.750000,5.475000,6.650000,-4.425000,-0.562500,-0.455000
50%,26.000000,1998.000000,18.000000,12.000000,12.000000,1.000000,0.000000,1.000000,1.000000,0.000000,...,18.000000,20.000000,-1.000000,-0.130000,-0.030000,17.950000,18.950000,-0.400000,-0.060000,0.015000
75%,29.000000,2001.000000,27.000000,21.500000,20.600000,2.000000,2.000000,4.000000,2.000000,0.000000,...,34.000000,32.000000,4.000000,0.500000,0.640000,31.150000,30.550000,2.600000,0.350000,0.412500
max,39.000000,2007.000000,34.000000,34.000000,34.000000,20.000000,12.000000,31.000000,19.000000,5.000000,...,82.000000,75.000000,56.000000,22.500000,21.320000,70.900000,71.100000,43.400000,5.540000,29.830000


In [4]:
import pandas as pd
data = pd.read_csv('Complete 2023-2024_Premier_League_Player_Stats_and_Wages.csv')
data['Pos'] = data['Pos'].apply(lambda x: x.split(',')[0])